In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import heapq as hp

df = pd.read_csv(u'data/iris.txt', sep=' ')

A = np.hstack([np.matrix(df.sl).T, np.matrix(df.sw).T, np.matrix(df.pl).T, np.matrix(df.pw).T])
c = np.matrix(df.c).T


# Define an object and overload custom comparison operators
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val, self.idx)


def get_maximum_occurrence(heap, c):
    categories = []
    for t in range(len(heap)):
        h = hp.heappop(heap)
        print("Neighbor %d" % (t + 1))
        print(h)
        categories.append(int(c[h.idx]))
    return max(set(categories), key=categories.count)
        

# Search for the nearest K
def predict_category_with_k_NN(K, A, test, c):
    heap = []
    N = A.shape[0]   
    
    # Fill in the heap with dummy nodes
    for i in range(K):
        hp.heappush(heap, tup(np.inf, -1))
    
    # Populate the heap with k nearest items
    for i in range(N):
        e = A[i, :] - test
        e = e.reshape((4, 1))
        tp = tup(float(e.T * e), i)
        if tp <= heap[0]:
            hp.heapreplace(heap, tp)
            
    return get_maximum_occurrence(heap, c)

test = np.mat([1.8, 2.1, 1.3, 1.2])
result = predict_category_with_k_NN(3, A, test, c)
print("\n")
print("Result category: %d" % result)

Neighbor 1
8.41,8
Neighbor 2
8.31,13
Neighbor 3
8.14,41


Result category: 1
